In [1]:
#import libraries
import functions as func
from joblib import Parallel, delayed
import numpy as np
import pandas as pd

#load preprocessed data
#df = func.getUseableHP2Data()

In [62]:
#read data
df = pd.read_csv("houseprices2.csv")

#get all column data types, create dict from colname to dtype
datatypes = [x for x in df.dtypes]
colnames = df.dtypes.index
dict_colname_to_dtype = dict(zip(colnames, datatypes))

#get cunts of na values per column
na_counts = [x for x in df.isna().sum()]

#handle na vals
for i in range(len(colnames)):
    column_name = colnames[i]
    column_type = dict_colname_to_dtype.get(column_name)
    column_num_na = na_counts[i]

#for categorical columns, create na category
    if column_num_na > 10 and column_type == "object":
        categotical_column = df[column_name]
        fixed_column = np.where(categotical_column.isna(), "NotApplicable", categotical_column)
        df[column_name] = fixed_column

#drop any non categorical where more than 10% of data missing
    if column_num_na > 150 and column_type != "object":
        df.drop(column_name, axis = 1, inplace = True)
            
        
print(len(df))
df.dropna(inplace=True)
print(len(df))
df.head

1460
1370


<bound method NDFrame.head of         Id  MSSubClass MSZoning  LotArea Street          Alley LotShape  \
0        1          60       RL     8450   Pave  NotApplicable      Reg   
1        2          20       RL     9600   Pave  NotApplicable      Reg   
2        3          60       RL    11250   Pave  NotApplicable      IR1   
3        4          70       RL     9550   Pave  NotApplicable      IR1   
4        5          60       RL    14260   Pave  NotApplicable      IR1   
...    ...         ...      ...      ...    ...            ...      ...   
1455  1456          60       RL     7917   Pave  NotApplicable      Reg   
1456  1457          20       RL    13175   Pave  NotApplicable      Reg   
1457  1458          70       RL     9042   Pave  NotApplicable      Reg   
1458  1459          20       RL     9717   Pave  NotApplicable      Reg   
1459  1460          20       RL     9937   Pave  NotApplicable      Reg   

     LandContour Utilities LotConfig  ... PoolArea         PoolQC  \


In [59]:
df = pd.read_csv("houseprices2.csv")
len(df)

1460

In [8]:
df = pd.read_csv("houseprices2.csv")
columns = list(df.columns)
x = df.isna().sum()
for i in range(len(x)):
    if x[i] > 50:
        print(columns[i])

LotFrontage
Alley
FireplaceQu
GarageType
GarageYrBlt
GarageFinish
GarageQual
GarageCond
PoolQC
Fence
MiscFeature


In [ ]:
# TODO: fix na situation for houseprices 2
df = pd.read_csv("houseprices2.csv")

pool = df["PoolQC"].tolist()
fence = df["Fence"].tolist()
misc = df["MiscFeature"].tolist()

pool2 = []
fence2 = []
misc2 = []
for i in range(len(df)):
    try:
        if np.isnan(pool[i]):
            pool2.append("NotApplicable")
        else:
            pool2.append(pool[i])
    except:
        pool2.append(pool[i])

    try:
        if np.isnan(fence[i]):
            fence2.append("NotApplicable")
        else:
            fence2.append(fence[i])
    except:
        fence2.append(fence[i])

    try:
        if np.isnan(misc[i]):
            misc2.append("NotApplicable")
        else:
            misc2.append(misc[i])
    except:
        misc2.append(misc[i])


df["PoolQC"] = pool2
df["Fence"] = fence2
df["MiscFeature"] = misc2

print(len(df))
df.dropna(inplace=True)
print(len(df))

Baseline model

In [ ]:
#fit and score full baseline model
X = df.copy(deep=True)
X.drop(["SalePrice"], axis=1, inplace=True)
y = df["SalePrice"]
base_mae = func.trainLinearRegression(X, y)
print(base_mae)

Variable selection model

In [ ]:
#get first new predictor
colnames = list(df.columns)
colnames.remove("SalePrice")
colnames = [[x] for x in colnames]
results = Parallel(n_jobs=6)(delayed(func.quickScoreLinearRegression)(i, True) for i in colnames)
first_new_colname = colnames[np.argmax(results)]
print(f"Therefore we now add {first_new_colname} as a model feature")

In [ ]:
#get second new predictor
colnames2 = list(df.columns)
colnames2.remove("SalePrice")
colnames2.remove(first_new_colname)
collnames2 = [[x, first_new_colname] for x in colnames2]
results = Parallel(n_jobs=6)(delayed(func.quickScoreLinearRegression)(i, True) for i in colnames2)
second_new_colname = colnames2[np.argmax(results)]
print(f"Therefore we now add {second_new_colname} as a model feature")

In [ ]:
#get third and final new predictor
colnames3 = list(df.columns)
colnames3.remove("SalePrice")
colnames3.remove(first_new_colname)
colnames3.remove(second_new_colname)
colnames3 = [[x, first_new_colname, second_new_colname] for x in colnames3]
results = Parallel(n_jobs=6)(delayed(func.quickScoreLinearRegression)(i, True) for i in colnames)
third_new_colname = colnames3[np.argmax(results)]
print(f"Finally we add {third_new_colname} as a model feature")
print(f"This gives our model a MAE of {min(results)}.")

Random forest regressor

In [2]:
rf_mae = func.quickScoreRandomForest()
print(rf_mae)

35187.0
